# **Exploring Pandas - Part 4**

# **Module 6: Merging, Joining, & Concatenation**
* 6.1. Exploring merge()
* 6.2. Exploring concat()
* 6.3. Exploring join()
* 6.4 Advanced Merging




In [ ]:
import pandas as pd

# Load DataSets
customers_df = pd.read_csv("customers.csv")
orders_df = pd.read_csv("orders.csv", parse_dates=["OrderDate"])

# Preview the Data
print("Customers:")
print(customers_df, "\n")

print("Orders:")
print(orders_df)


Customers:
    CustID      Name Region       City
0        1  Srinivas   East    Kolkata
1        2       Vas   West     Mumbai
2        3     Hello  North      Delhi
3        4    Dandes  South    Chennai
4        5        OK   East    Kolkata
5        6       Hai   West       Pune
6        7        SD  North     Jaipur
7        8        DS  South  Hyderabad
8        9    Manish   West       Pune
9       10      Amit  North     Jaipur
10      11      Alok  South  Hyderabad 

Orders:
    OrderID  CustID  OrderDate  Amount
0       101       1 2025-01-05     250
1       102       1 2025-02-10     450
2       103       2 2025-01-15     300
3       104       4 2025-03-01     500
4       105       6 2025-02-20     700
5       106       3 2025-01-22     150
6       107       3 2025-03-14     375
7       108       5 2025-03-29     620
8       109       8 2025-01-07     210
9       110       2 2025-01-07     830
10      111       7 2025-05-09     400
11      112       7 2025-06-12     520
12  

## **6.1 Exploring merge()**
* merge() function in Pandas is similar to SQL JOIN.
* It combines rows from two DataFrames based on a common key (or index).

**Syntax:**



```
pd.merge(left_df, right_df, how="inner", on="key_column")

```



### **A) Inner Join (default)**
* Rows are included only when the key exists in both DataFrames.

**Use Case:**
* Get only customers who placed orders.


In [ ]:
# Inner Join (default)
inner_merge = pd.merge(customers_df, orders_df, on="CustID", how="inner")

inner_merge

In [ ]:
# From our datasets
myresults = pd.merge(customers_df, orders_df, on="CustID", how="inner", validate="one_to_many" )
myresults

### **B) Left Join:**
* Keeps all rows from left DataFrame (customers_df).
* Missing orders → NaN.

**Use Case:**
* Get all customers and their orders (if any).


In [ ]:
# Left Join
left_merge = pd.merge(customers_df, orders_df, on="CustID", how="left")

left_merge

### **C) Right Join:**
* Keeps all rows from right DataFrame (orders_df).
* Missing customers → NaN.

**Use Case:**
* Get all orders and attach customer info.


In [ ]:
# Right Join
right_merge = pd.merge(customers_df, orders_df, on="CustID", how="right")

right_merge

### **D) Outer Join:**
* Keeps rows from both DataFrames.
* Missing values filled with NaN.

**Use Case:**
* Get the full picture — all customers + all orders.



In [ ]:
# Outer Join
outer_merge = pd.merge(customers_df, orders_df, on="CustID", how="outer")

outer_merge

### **E) Types of Relationships in Pandas Merge**
* We can establish 3 types of Relationship between two DataFrames being merged
  * one_to_one
  * one_to_many
  * many-to-many




**1) One-to-One Relationship**
* Each key in the left DataFrame matches with at most one key in the right DataFrame.
* Each key appears only once in both DataFrames.
* If any duplicate keys exist, validate="one_to_one" will raise an error.

In [ ]:
import pandas as pd

mycustomers_df = pd.DataFrame({
    "CustID": [1, 2, 3],
    "Name": ["Srinivas", "Hello", "SD"]
})

myorders_df = pd.DataFrame({
    "CustID": [1, 2, 3],
    "OrderID": [101, 102, 103]
})

# One-to-One
result = pd.merge(
    mycustomers_df, myorders_df,
    on="CustID", validate="one_to_one"
)

result


,CustID,Name,OrderID
0,1,Srinivas,101
1,2,Hello,102
2,3,SD,103


**2) One-to-Many Merge**
* Keys in the left DataFrame are unique, but keys in the right DataFrame may repeat.
* This is the most common case (customers placing multiple orders).
* **validate="one_to_many"**
  * Ensures left keys are unique, but allows right duplicates.

In [ ]:

mycustomers_df = pd.DataFrame({
    "CustID": [1, 2, 3],
    "Name": ["Srinivas", "Hello", "SD"]
})

myorders_df = pd.DataFrame({
    "CustID": [1, 1, 2, 3, 3],
    "OrderID": [201, 202, 203, 204, 205]
})

# One-to-Many
result = pd.merge(
    mycustomers_df, myorders_df,
    on="CustID", validate="one_to_many"
)

result

,CustID,Name,OrderID
0,1,Srinivas,201
1,1,Srinivas,202
2,2,Hello,203
3,3,SD,204
4,3,SD,205


**3. Many-to-Many Relationship**
* Both DataFrames contain duplicate keys.

In [ ]:
import pandas as pd

# Step 1 – Students

students_df = pd.DataFrame({
    "StudentID": [1, 2, 3, 4, 5],
    "StudentName": ["Sri", "Vas", "Hello", "Sd", "Ds"]
})

# Step 2 – Courses

courses_df = pd.DataFrame({
    "CourseID": [101, 102, 103],
    "CourseName": ["Java", "ML", "Python"]
})

# Step 3 – Enrollment (bridge table)

enrollment_df = pd.DataFrame({
    "StudentID": [1, 1, 2, 2, 3, 4, 5, 5],
    "CourseID":  [101, 102, 101, 103, 102, 103, 101, 103]
})

# Step 4 – Merge enrollment_df with students_df

enrollment_students = pd.merge(
    enrollment_df, students_df,
    on="StudentID",
    validate="many_to_one"
)

print(enrollment_students)

# Step 5 – Merge the result with courses_df

final_result = pd.merge(
    enrollment_students, courses_df,
    on="CourseID",
    validate="many_to_one"
)

final_result


## **6.2. Exploring concat()**
* concat() function in Pandas is used to combine DataFrames along rows or columns.
* Unlike merge(), it does not use a common key — instead, it simply stacks or joins objects along a given axis.
* **Think of it as:**
  * **Row-wise (axis=0):** place one below the other.
  * **Column-wise (axis=1):** place side by side.


**Syntax:**



```
pd.concat([df1, df2], axis=0)   # Row-wise (default)
pd.concat([df1, df2], axis=1)   # Column-wise

```



In [ ]:
import pandas as pd

# Load subsets
orders_q1 = pd.read_csv("orders_q1.csv", parse_dates=["OrderDate"])
orders_q2 = pd.read_csv("orders_q2.csv", parse_dates=["OrderDate"])

print(orders_q1)

print("-"*36)

print(orders_q2)


   OrderID  CustID  OrderDate  Amount
0      101       1 2025-01-05     250
1      102       1 2025-02-10     450
2      103       2 2025-01-15     300
3      104       4 2025-03-01     500
4      105       6 2025-02-20     700
5      106       3 2025-01-22     150
6      107       3 2025-03-14     375
7      108       5 2025-03-29     620
8      109       8 2025-01-07     210
------------------------------------
   OrderID  CustID  OrderDate  Amount
0      110       2 2025-04-18     830
1      111       7 2025-05-09     400
2      112       7 2025-06-12     520
3      113      10 2025-04-01     999


### **A) Row-wise Concatenation (axis=0)**
* Stacks DataFrames vertically (one below the other).
* Useful when data is split by time, region, or source and needs to be rebuilt.




**Use Case 1: Combine 2 Dataframes.**
* Combine Q1 and Q2 orders into a single dataset

In [ ]:
all_orders = pd.concat([orders_q1, orders_q2], axis=0)

print(all_orders.shape)    # total rows = Q1 + Q2

all_orders = all_orders.sort_values("OrderDate")

all_orders

(13, 4)


,OrderID,CustID,OrderDate,Amount
0,101,1,2025-01-05,250
8,109,8,2025-01-07,210
2,103,2,2025-01-15,300
5,106,3,2025-01-22,150
1,102,1,2025-02-10,450
4,105,6,2025-02-20,700
3,104,4,2025-03-01,500
6,107,3,2025-03-14,375
7,108,5,2025-03-29,620
3,113,10,2025-04-01,999


**Use Case 2: Reset Index.**
* Clean up the index after stacking.

In [ ]:
all_orders_clean = pd.concat([orders_q1, orders_q2], axis=0, ignore_index=True)

all_orders_clean

**Use Case 3: Track Source with Keys**
* Keep track of which quarter each order came from.


In [ ]:
orders_with_keys = pd.concat([orders_q1, orders_q2], axis=0, keys=["Q1", "Q2"])

orders_with_keys

# This creates a MultiIndex:
# Level 1 → Source key (Q1 or Q2)
# Level 2 → Original row index

#### **Access subset by key**

**1. Access all rows for a key (loc with string)**

In [ ]:
# All Q1 rows
orders_with_keys.loc["Q1"]

In [ ]:
# All Q2 rows
orders_with_keys.loc["Q2"]

**2. Access multiple keys (list of labels)**

In [ ]:
# Returns both Q1 and Q2
orders_with_keys.loc[["Q1", "Q2"]]

**3. Access rows from a specific key + row index**

In [ ]:
# Row 0 from Q1
orders_with_keys.loc[("Q1", 0)]


In [ ]:
# Row 5 from Q2
orders_with_keys.loc[("Q2", 3)]

**4. Slice within a key**

In [ ]:
# First 3 rows of Q1
orders_with_keys.loc["Q1"].head(3)


In [ ]:
# Last 2 rows of Q2
orders_with_keys.loc["Q2"].tail(2)

### **B) Column-wise Concatenation (axis=1)**
* Stacks DataFrames side by side (adds new columns).
* Useful when you have different attributes for the same set of rows




**Use Case 1: Combine 2 Dataframes**
* Split customers into two DataFrames (personal info vs. location), then concat back.

In [ ]:

# Load customers
customers_df = pd.read_csv("customers.csv")
print(customers_df)
print("-"*36)

# Split into two smaller DataFrames
cust_info = customers_df[["CustID","Name"]]
print(cust_info)
print("-"*36)

cust_region = customers_df[["Region","City"]]
print(cust_region)
print("-"*36)

# Column-wise concat
customers_full = pd.concat([cust_info, cust_region], axis=1)
customers_full

**Use Case 2: Handling Mismatched Indexes**
* When concatenating DataFrames with different indexes (or different row counts), Pandas aligns by index.
* Any missing match → NaN.
* This happens in column-wise concat (axis=1)


In [ ]:
customers_df = pd.read_csv("customers.csv")

# Split into two DataFrames with different lengths
cust_info = customers_df[["CustID","Name"]]        # all 11 rows
cust_region = customers_df.loc[0:4, ["Region","City"]]  # only first 5 rows

# Column-wise concat with mismatched indexes
myresults = pd.concat([cust_info, cust_region], axis=1)

myresults

### **C) Handling Mismatched Indexes in pd.concat()**
* When concatenating DataFrames with different indexes (or different row counts), Pandas aligns by index.
* Any missing match → NaN.
* This happens in column-wise concat (axis=1)

In [ ]:
import pandas as pd

customers_df = pd.DataFrame({
    "CustID": ["C1","C2","C3","C4","C5","C6","C7","C8"],
    "Name":   ["A","B","C","D","E","F","G","H"],
    "Region": ["South","North","East","West","South","North","East","West"],
    "City":   ["Guntur","Delhi","Kolkata","Mumbai","Chennai","Lucknow","Patna","Pune"]
})

# Left: pick NON-contiguous rows → index labels [1,3,4,6]
cust_info = customers_df.loc[[1,3,4,6], ["CustID","Name"]]
print(cust_info)

# Right: pick a different slice → index labels [0,1,2,3]
cust_region = customers_df.loc[[0,1,2,3], ["Region","City"]]
print(cust_region)


  CustID Name
1     C2    B
3     C4    D
4     C5    E
6     C7    G
  Region     City
0  South   Guntur
1  North    Delhi
2   East  Kolkata
3   West   Mumbai


**a) Customers Split Unevenly (Mismatched Indexes)**

In [ ]:
misaligned = pd.concat([cust_info, cust_region], axis=1)
print(misaligned)

  CustID Name Region     City
1     C2    B  North    Delhi
3     C4    D   West   Mumbai
4     C5    E    NaN      NaN
6     C7    G    NaN      NaN
0    NaN  NaN  South   Guntur
2    NaN  NaN   East  Kolkata


* For index 1,3, both DataFrames have rows → values combine fine.
* For index 4,6, cust_region has no rows → NaN in Region and City.
* For index 0,2, cust_info has no rows → NaN in CustID and Name.

**b) Reset Index to Fix Alignment (Row-order Alignment)**
* Force row-order alignment instead of index alignment.

In [ ]:
cust_info_reset = cust_info.reset_index(drop=True)
print(cust_info_reset)

cust_region_reset = cust_region.reset_index(drop=True)
print(cust_region_reset)

aligned = pd.concat([cust_info_reset, cust_region_reset], axis=1)
print(aligned)



  CustID Name
0     C2    B
1     C4    D
2     C5    E
3     C7    G
  Region     City
0  South   Guntur
1  North    Delhi
2   East  Kolkata
3   West   Mumbai
  CustID Name Region     City
0     C2    B  South   Guntur
1     C4    D  North    Delhi
2     C5    E   East  Kolkata
3     C7    G   West   Mumbai


### **D) concat(axis=1) Vs merge()**
**Use concat(axis=1) when:**
* You’re combining different features/columns for the same rows.
* Indexes are guaranteed aligned (e.g., splitting and rejoining the same DataFrame).

**Use merge() when:**
* You’re combining relational data (like Customers ↔ Orders).
* Keys may not match perfectly.
* You need join types (inner, left, right, outer).

## **6.3 Exploring join()**
* Easy way for combining DataFrames based on their index.
* Think of it as: “Align by index, and attach extra columns.”
* Default join = left join (keeps all rows of the caller).

**Syntax:**


```
DataFrame.join(other, how="left", on=None)

```

* **other** → DataFrame to join with.
* **how** → "left" (default), "right", "inner", "outer".
* **on** → Column in the caller to use as join key (rare; usually index is used).


In [ ]:
import pandas as pd

# Customers
customers_df = pd.DataFrame({
    "CustID": [1,2,3],
    "Name": ["Sri","Vas","Hello"]
})
print(customers_df)

# Orders
orders_df = pd.DataFrame({
    "CustID": [1,2,2,3],
    "OrderID": [101,102,103,104],
    "Amount": [250, 120, 450, 300]
})
print(orders_df)


   CustID   Name
0       1    Sri
1       2    Vas
2       3  Hello
   CustID  OrderID  Amount
0       1      101     250
1       2      102     120
2       2      103     450
3       3      104     300


In [ ]:

# Set CustID as index
cust_indexed_df = customers_df.set_index("CustID")
print(cust_indexed_df)

orders_indexed_df = orders_df.set_index("CustID")
print(orders_indexed_df)


         Name
CustID       
1         Sri
2         Vas
3       Hello
        OrderID  Amount
CustID                 
1           101     250
2           102     120
2           103     450
3           104     300


In [ ]:

# Join using index
joined = cust_indexed_df.join(orders_indexed_df, how="inner")
joined

,Name,OrderID,Amount
CustID,,,
1,Sri,101,250
2,Vas,102,120
2,Vas,103,450
3,Hello,104,300


## **6.4. join() vs merge()**

**merge()**
* Uses columns (explicit keys) to match rows.
* SQL-style joins supported: inner, left, right, outer.
* You must specify the on column (or left_on / right_on).
* Very flexible: can join on multiple keys, handle overlapping column names with suffixes, etc.
* Typical use case: relational/tabular data where you have foreign keys (e.g., CustID, OrderID).
* Rule of Thumb: Use merge() when working with relational-style datasets and want full control over join conditions.

**join()**
* Uses index (implicit keys) to align rows.
* Shorthand for index-based joins.
* Default join = left join (keeps all rows of caller).
* Less flexible than merge(): usually used for one key (the index).
* Typical use case: time-series or index-based data, or when you already set meaningful indexes.
* Rule of Thumb: Use join() when indexes are already your keys and you want a simpler syntax.


## **6.5 Advanced Merging**

### **A) Merging on Multiple Keys**
* Sometimes one column is not enough to uniquely identify a record.
* **In an e-commerce schema:**
  * Orders table → OrderID, CustID.
  * Order_Items table → each row defined by (OrderID, ProductID).
  * Products table → ProductID, ProductName, UnitPrice.

* To join Orders ↔ Order_Items ↔ Products, we need multiple keys.

**Step 1: Orders ↔ Order_Items**

In [ ]:
import pandas as pd

# Orders
orders_df = pd.DataFrame({
    "OrderID": [101, 102],
    "CustID": [1, 2],
    "OrderDate": pd.to_datetime(["2025-09-01", "2025-09-02"])
})

# Order items (multiple products per order)
order_items_df = pd.DataFrame({
    "OrderID": [101, 101, 102],
    "ProductID": [1001, 1002, 1003],
    "Quantity": [2, 1, 5]
})


# Merge Orders with Order_Items
orders_items = pd.merge(orders_df, order_items_df, on="OrderID", how="inner")

orders_items


**Step 2:  Merge with Products (on ProductID)**

In [ ]:
# Products
products = pd.DataFrame({
    "ProductID": [1001, 1002, 1003],
    "ProductName": ["Laptop", "Mouse", "Keyboard"],
    "UnitPrice": [60000, 800, 1500]
})

orders_full = pd.merge(orders_items, products, on="ProductID", how="inner")

orders_full


In [ ]:

# Add line total
orders_full["LineTotal"] = orders_full["Quantity"] * orders_full["UnitPrice"]

orders_full

**Step 3: Order Totals**

In [ ]:
order_totals = orders_full.groupby("OrderID")["LineTotal"].sum().reset_index()

order_totals

### **B) left_on and right_on (Different Column Names)**
* Sometimes the join keys have different names in the two DataFrames.
* merge() allows you to explicitly specify them with left_on and right_on.
* Very common when merging external / third-party datasets.



**a) UseCase: Customers vs Customer_Alias**
  * customers_df → key is CustID
  * customer_alias → key is CustomerID

* We want to join them even though column names don’t match.

In [ ]:
# Customers
customers_df = pd.DataFrame({
    "CustID": [1, 2, 3],
    "Name": ["Sri", "Vas", "Hello"]
})

# Alias (different column name)
customer_alias = pd.DataFrame({
    "CustomerID": [1, 2, 3],
    "Alias": ["S1", "V1", "H1"]
})


# Merge with different column names
merged_alias = pd.merge(
    customers_df, customer_alias,
    left_on="CustID", right_on="CustomerID",
    how="inner"
)

merged_alias

**b) Drop duplicate CustomerID column**

In [ ]:
merged_alias = merged_alias.drop(columns=["CustomerID"])
merged_alias

,CustID,Name,Alias
0,1,Sri,S1
1,2,Vas,V1
2,3,Hello,H1


### **C) Handling Overlapping Columns with suffixes**
* When both DataFrames have the same non-key column names, Pandas adds _x and _y.
* Use suffixes to make labels clear (e.g., old vs new).



**A) Default behavior (auto _x / _y)**

In [ ]:
# Customers (old info)
customers_df = pd.DataFrame({
    "CustID": [1,2],
    "Name": ["Sri","Vas"],
    "Email":["sri@jlc.com","vas@jlc.com"],
    "City": ["Delhi","Mumbai"]
})

# Customer updates (new info)
customer_updates = pd.DataFrame({
    "CustID": [1,2],
    "Email":["sri@da.com","vas@da.com"],
    "City": ["Bangalore","Pune"]
})

# Merge
merged = pd.merge(
    customers_df, customer_updates,
    on="CustID", how="inner")


merged

,CustID,Name,Email_x,City_x,Email_y,City_y
0,1,Sri,sri@jlc.com,Delhi,sri@da.com,Bangalore
1,2,Vas,vas@jlc.com,Mumbai,vas@da.com,Pune


**B) Use custom suffixes to make intent clear**

In [ ]:
# Merge with suffixes
merged = pd.merge(
    customers_df, customer_updates,
    on="CustID", how="inner",
    suffixes=("_old", "_new")
)

merged

# **Module 7: Advanced DataFrame Reshaping**
* 7.1 MultiIndexing
* 7.2 stack() & unstack()
* 7.3 melt() & pivot()




## **7.1 MultiIndexing (Hierarchical Indexing)**
* MultiIndex allows multiple levels of indexing (rows or columns).
* Think of it like nested keys → (Region, Quarter) or (Person, Course).
* Makes reshaping, slicing, and aggregations much more powerful.


### **A) Creating a MultiIndex**
* Using set_index() with columns of Dataframe


In [ ]:
import pandas as pd

orders_products = pd.read_csv("orders_products.csv", parse_dates=["OrderDate"])

orders_products


In [ ]:

# Set MultiIndex (Region, Quarter)
myorders = orders_products.set_index(["Region","Quarter"])

myorders

### **B) Accessing MultiIndex Data**




In [ ]:
# Select all rows for a single (Region, Quarter)
myorders.loc[("North","Q1")]


In [ ]:
# Select all rows for a single (Region, Quarter)
myorders.loc[("North","Q2")]


In [ ]:
# Select all rows for a given Region (all Quarters)
myorders.loc["West"]


In [ ]:
# Select multiple keys (list inside tuple)
myorders.loc[("North", ["Q1","Q2"]), :]


In [ ]:
# Select multiple Regions with slice
myorders.loc[["East","South"]]


In [ ]:
myorders.loc[["East","South"]]["LineTotal"]

In [ ]:
myorders.loc[["East","South"]]["LineTotal"].sum()

np.int64(3760)

### **C) Reordering & Sorting Levels**


In [ ]:
# Swap levels
swapped = myorders.swaplevel("Region","Quarter")
swapped


In [ ]:
# Sort by index
sorted_orders = swapped.sort_index()

sorted_orders

### **D) Resetting Index**


In [ ]:
# Bring MultiIndex levels back as columns
reset = myorders.reset_index()
reset

## **7.2 stack() & unstack()**
* stack() → moves columns → rows (longer format).
* unstack() → moves rows → columns (wider format).
* They work best with MultiIndex DataFrames.


### **A) Simple Example**


In [ ]:

import pandas as pd

students_df = pd.DataFrame({
    "Person": ["Sri","Sri","Sri","Vas","Vas","Vas","SD","SD","SD"],
    "Course": ["Math","Science","English","Math","Science","English","Math","Science","English"],
    "Score": [85, 90, 78, 88,80,90,50,56,57]
})

students_df

,Person,Course,Score
0,Sri,Math,85
1,Sri,Science,90
2,Sri,English,78
3,Vas,Math,88
4,Vas,Science,80
5,Vas,English,90
6,SD,Math,50
7,SD,Science,56
8,SD,English,57


**1. Set MultiIndex**
* Index the Rows by Person → Course.

In [ ]:
mydf1 = students_df.set_index(["Person","Course"])

mydf1

Score
Person Course        
Sri    Math        85
       Science     90
       English     78
Vas    Math        88
       Science     80
       English     90
SD     Math        50
       Science     56
       English     57

**2. Unstack (rows → columns)**
* Move the Course to columns, giving a wide format.

In [ ]:
mydf2 = mydf1.unstack(level="Course")

mydf2

Score             
Course English Math Science
Person                     
SD          57   50      56
Sri         78   85      90
Vas         90   88      80

**3. Stack (columns → rows)**
* Brings it back to long format.

In [ ]:
mydf3 = mydf2.stack()

mydf3


In [ ]:
mydf4 = mydf3.reset_index()
mydf4

## **7.3 melt() & pivot()**


### **A) melt() : Wide → Long**
* Converts multiple columns into rows (tidy format).
* Useful when you have repeated attributes like months or years in separate columns.

In [ ]:
import pandas as pd

# wide-format sales data
sales_wide = pd.DataFrame({
    "Region": ["East","West","North","South"],
    "Jan": [250,200,180,300],
    "Feb": [300,220,190,310],
    "Mar": [280,210,200,320]
})

sales_wide


In [ ]:
# Melt into Long Format

sales_long = pd.melt(
    sales_wide,
    id_vars=["Region"],              # keep Region fixed
    value_vars=["Jan","Feb","Mar"],  # melt these columns
    var_name="Month",                # new column for old headers
    value_name="Sales"               # new column for values
)

sales_long

### **B) pivot(): Long → Wide**
*  Opposite of melt() → converts rows back into columns.

In [ ]:
# Convert back to Wide Format

sales_pivot = sales_long.pivot(
    index="Region",   # rows
    columns="Month",  # new column headers
    values="Sales"    # fill values
)

sales_pivot


# **Module 8: Time Series Handling**
* 8.1 Working with Datetime in Pandas
* 8.2 DateTime Indexing & Slicing




## **8.1. Working with Datetime in Pandas**



### **A) Convert Date column into datetime**
*  Using pd.to_datetime()

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("time_series_sales.csv")

# Convert 'Date' column to datetime
df["Date"] = pd.to_datetime(df["Date"])

# Check datatype
print(df.dtypes)
print("-"*30)

df.head(10)

### **B) Extract datetime components**
* Extract year, month, day, weekday

In [ ]:
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day
df["Weekday"] = df["Date"].dt.day_name()

print(df.head(10))

### **C) Create a date range**
* Create a range of dates (weekly frequency)

In [5]:
date_range = pd.date_range(start="2023-01-01", end="2023-03-31", freq="W")

date_range


DatetimeIndex(['2023-01-31', '2023-02-28', '2023-03-31'], dtype='datetime64[ns]', freq='ME')

* Create a range of dates (Monthly frequency)

In [8]:
date_range = pd.date_range(start="2023-01-01", end="2023-02-28", freq="ME")

date_range


DatetimeIndex(['2023-01-01', '2023-01-08', '2023-01-15', '2023-01-22',
               '2023-01-29', '2023-02-05', '2023-02-12', '2023-02-19',
               '2023-02-26'],
              dtype='datetime64[ns]', freq='W-SUN')

* Create a range of dates (Quaterly frequency)

In [9]:
date_range = pd.date_range(start="2023-01-01", end="2023-09-30", freq="QE")

date_range


DatetimeIndex(['2023-03-31', '2023-06-30', '2023-09-30'], dtype='datetime64[ns]', freq='QE-DEC')

### **D) Handling Different Date Formats**
* Sometimes dates come in DD-MM-YYYY instead of YYYY-MM-DD.


In [18]:
# dataset with different format
mydf = pd.DataFrame({
    "Date": ["01-03-2023", "15-03-2023", "28-03-2023"],  # DD-MM-YYYY
    "Sales": [200, 350, 500]
})

# Convert using format and dayfirst
mydf["Date"] = pd.to_datetime(mydf["Date"], format="%d-%m-%Y")

mydf
# format="%d-%m-%Y" ensures Pandas interprets correctly.


,Date,Sales
0,2023-03-01,200
1,2023-03-15,350
2,2023-03-28,500


### **E) Handling Invalid Dates**
* Sometimes datasets contain invalid dates like "2023-02-30".


In [23]:
# Example dataset with invalid dates
mydf = pd.DataFrame({
    "Date": ["2023-02-28", "2023-02-30", "2023-03-05","10-10-2023"],
    "Sales": [250, 400, 300, 350]
})

# Convert with errors="coerce" → invalid dates become NaT (Not a Time)
mydf["Date"] = pd.to_datetime(mydf["Date"], errors="coerce")

mydf

# The invalid date "2023-02-30" will turn into NaT.

,Date,Sales
0,2023-02-28,250
1,NaT,400
2,2023-03-05,300
3,NaT,350


### **F) Handling Time in Datetime**
* You can extract hours, minutes, seconds using dt.hour, dt.minute, dt.second.

In [21]:
df_time = pd.DataFrame({
    "DateTime": ["2023-03-01 14:30:23", "2023-03-01 18:45:56"],
    "Sales": [120, 220]
})

df_time["DateTime"] = pd.to_datetime(df_time["DateTime"])

# Extract components
df_time["Hour"] = df_time["DateTime"].dt.hour       # 14, 18
df_time["Minute"] = df_time["DateTime"].dt.minute   # 30, 45
df_time["Second"] = df_time["DateTime"].dt.second   # 23, 56

df_time

,DateTime,Sales,Hour,Minute,Second
0,2023-03-01 14:30:23,120,14,30,23
1,2023-03-01 18:45:56,220,18,45,56


## **8.2. DateTime Indexing & Slicing**

**1) Load, convert, set, and sort**
* Use time_series_sales dataset

In [24]:
import pandas as pd

df = pd.read_csv("time_series_sales.csv")

df["Date"] = pd.to_datetime(df["Date"], errors="coerce",format="%Y-%m-%d")

df = df.set_index("Date").sort_index()

df.head()


,Sales,Event
Date,,
2023-01-01,152,Promo
2023-01-02,485,Normal
2023-01-03,398,Promo
2023-01-04,320,Normal
2023-01-05,156,Normal


**2) Partial-string indexing**
*  Select by year, month, or exact day.

In [ ]:
# All of February 2023
feb_2023 = df.loc["2023-02"]

feb_2023

In [26]:
# A specific day (if present)
day_2023_03_15 = df.loc["2023-03-15"]

day_2023_03_15

,2023-03-15
Sales,130
Event,Normal


In [ ]:
# All of 2023 (works even if multiple years exist)
year_2023 = df.loc["2023"]

year_2023.head(10)

**3) Range slicing (inclusive)**

In [ ]:
# January 10 to January 20 (both included)
jan_window = df.loc["2023-01-10":"2023-01-20"]

jan_window

In [ ]:
# Feb 10 to Feb 20
feb_window = df.loc["2023-02-10":"2023-02-15"]

feb_window

**3) Monthly Slice**
* Shorthand for a whole month.


In [ ]:
# All rows for March 2023
march_2023 = df.loc["2023-03"]

march_2023

**4) Boolean Filters on Datetime Components**
* Filter using weekday, month, or custom conditions.

In [ ]:
# Only Mondays (any month)
mondays = df[df.index.day_name() == "Monday"]

mondays

In [ ]:
# Only March rows
march = df[df.index.month == 3]

march

In [ ]:
# Fridays in February 2023
feb_sales = df.loc["2023-02"]
fridays_feb = feb_sales[feb_sales.index.day_name() == "Friday"]

fridays_feb

**5) Combined Conditions**

In [ ]:
# March Sales
march_sales = df.loc["2023-03"]

march_sales

In [ ]:
# Promo days in March

promo_march = march_sales[(march_sales["Event"] == "Promo")]

promo_march

In [ ]:
# Promo days in March with Sales > 250

promo_march_high = promo_march[(promo_march["Event"] == "Promo") & (promo_march["Sales"] > 250)]

promo_march_high

In [ ]:
# Normal days in Feb with Sales > 400
mydf = df.loc["2023-02"]
promo_feb_high = mydf[(mydf["Event"] == "Normal") & (mydf["Sales"] >= 400)]

promo_feb_high

# **Module 9: Exporting & Saving Data**
* 9.1. Save to CSV
* 9.2. Appending to existing files

## **9.1. Save to CSV**
* Universally supported — works with Excel, SQL databases, Python, R, etc.
* Most common format for data exchange and EDA.

* Pandas method: **to_csv().**

In [43]:
import pandas as pd

# Sample student dataset
students = pd.DataFrame({
    "ID": [1, 2, 3, 4, 5],
    "Name": ["Sri", "Vas", "Hello", "SD", "DS"],
    "Marks": [85, 90, 78, 92, 88],
    "Grade": ["A", "A", "B", "A+", "A"]
})

students

,ID,Name,Marks,Grade
0,1,Sri,85,A
1,2,Vas,90,A
2,3,Hello,78,B
3,4,SD,92,A+
4,5,DS,88,A


In [46]:
#Write to CSV
students.to_csv("mystudents.csv", index=False)


**Useful Parameters in to_csv()**

**1) Delimiter**
* Save as tab-separated instead of comma-separated.

```
students.to_csv("students.tsv", sep="\t", index=False)
```

**2) Encoding**
* Handle non-English characters safely.

```
students.to_csv("students_utf8.csv", encoding="utf-8", index=False)
```

**3) Compression**
* Save large files in compressed format.

```
students.to_csv("students.csv.gz", compression="gzip", index=False)
```

**4) Select Columns**
* Save only specific columns.

```
students.to_csv("students_subset.csv", columns=["ID","Name"], index=False)
```

**5) Missing Values Representation**
* Replace NaN with a custom string when saving.

```
students.to_csv("students_na.csv", na_rep="MISSING", index=False)
```




In [47]:
import numpy as np
import pandas as pd

# Sample student dataset
students = pd.DataFrame({
    "ID": [1, 2, 3, 4, 5],
    "Name": ["Sri", "Vas", "Hello", np.nan, "DS"],
    "Marks": [85, 90, 78, np.nan, 88],
    "Grade": ["A", "A", "B", "A+", np.nan]
})

students
students.to_csv("students.csv", na_rep="MISSING", columns=["ID","Name","Marks"], index=False)



## **9.2 Appending to Existing Files**
* Sometimes we don’t want to overwrite a CSV file, but add new rows at the end.
* Use mode="a" → append mode.
* Use header=False → prevents writing column names again.

**Step 1: Save the initial file (with header)**

In [48]:
import pandas as pd

# Sample student dataset
students = pd.DataFrame({
    "ID": [1, 2, 3, 4, 5],
    "Name": ["Sri", "Vas", "Hello", "SD", "DS"],
    "Marks": [85, 90, 78, 92, 88],
    "Grade": ["A", "A", "B", "A+", "A"]
})

students

students.to_csv("mystudents123.csv", index=False)

**Step 2: Create new students DataFrame**

In [50]:
new_students = pd.DataFrame({
    "ID": [6,7],
    "Name": ["AAAAA","BBBBB"],
    "Marks": [80,95],
    "Grade": ["B","A+"]
})

new_students

,ID,Name,Marks,Grade
0,6,AAAAA,80,B
1,7,BBBBB,95,A+


**Step 3: Append to the same CSV (without header)**

In [51]:
new_students.to_csv("mystudents123.csv", mode="a", header=False, index=False)


**Key Points:**
* First write normally → header included.
* For subsequent appends → mode="a", header=False.


# **Module 10: Handling Large Files in Chunks**
* 10.1. Problem with Big Files
* 10.2. Reading in Chunks


## **10.1. Problem with Big Files**
* Normally, we read the file as follows

```
df = pd.read_csv("big.csv")
```

* Loads the entire file into memory (RAM).
* Can crash if file size > available memory.

* Solution:
  * Read/process data in smaller pieces = chunks.
  * Chunking streams the file in small DataFrame pieces so you process gradually.



## **10.2. Reading in Chunks**


**a)  Quick start: read in chunks**

In [52]:
import pandas as pd

# Read file in chunks of 10 rows
for chunk in pd.read_csv("time_series_sales.csv", chunksize=10):
    print("Chunk shape:", chunk.shape, type(chunk))


Chunk shape: (10, 3) <class 'pandas.core.frame.DataFrame'>
Chunk shape: (10, 3) <class 'pandas.core.frame.DataFrame'>
Chunk shape: (10, 3) <class 'pandas.core.frame.DataFrame'>
Chunk shape: (10, 3) <class 'pandas.core.frame.DataFrame'>
Chunk shape: (10, 3) <class 'pandas.core.frame.DataFrame'>
Chunk shape: (10, 3) <class 'pandas.core.frame.DataFrame'>
Chunk shape: (10, 3) <class 'pandas.core.frame.DataFrame'>
Chunk shape: (10, 3) <class 'pandas.core.frame.DataFrame'>
Chunk shape: (10, 3) <class 'pandas.core.frame.DataFrame'>


**b) Count rows without loading full file**

In [53]:
row_count = 0
for chunk in pd.read_csv("time_series_sales.csv", chunksize=10):
    row_count += len(chunk)

print("Total rows:", row_count)

Total rows: 90


**c) Filter while reading (stream → write)**

In [58]:
import pandas as pd

first = True
for mydf in pd.read_csv("time_series_sales.csv", chunksize=10, parse_dates=["Date"]):

    march_high_promo_sales = mydf[
        (mydf["Sales"] > 100) &                # high sales days
        (mydf["Event"] == "Promo") &           # only promo events
        (mydf["Date"].dt.month == 3)           # only March
    ]

    # Save incrementally
    march_high_promo_sales.to_csv("march_high_promo_sales.csv",mode="w" if first else "a", header=first, index=False)

    first = False

print("All Done")
print("Pandas Completed Successfully")


All Done
Pandas Completed Successfully
